In [1]:
import docx
import textract
from pptx import Presentation
import docx

import os
#from collections import defaultdict
import io
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import sys, getopt
import re

import nltk
import numpy as np
import random
import string
#from operator import add
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [2]:
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only
#nltk.download('maxent_ne_chunker') # first-time use only
#nltk.download('words')# first-time use only


In [3]:
def getFilesInAllSubFolder(folder):
    all_file_list = []
    with os.scandir(folder) as entries:
        for entry in entries:
            filename = os.path.join(folder, entry.name)
            abs_fname = os.path.abspath(filename)
            if os.path.isfile(abs_fname):
                #print(filename +" is a file")
                all_file_list.append(abs_fname)
            else:
                #print(filename +" is a folder")
                all_file_list = all_file_list + getFilesInAllSubFolder(filename)
    
    return all_file_list

In [4]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = re.sub('�', '', text)
    text = re.sub('\s+',' ',text)
    return text

In [5]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#text = "jeevitha\n how \nare you"
#LemNormalize(text)

In [6]:
# Used document similarity to generate response

def response(user_response, resultDict, sent_tokens):
    robo_response=''
    sent_tokens.append(user_response)
    #print("sent_tokens_all_files", sent_tokens)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    #print(idx)
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    sent_tokens.remove(user_response)
    #print("final dict", final_dict)
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you. Want to try something else?"
        return robo_response, None
    else:
        response_sen1 = sent_tokens[idx]
        robo_response = robo_response+sent_tokens[idx]+sent_tokens[idx+1]+sent_tokens[idx+2]+sent_tokens[idx+3]
        if response_sen1 in resultDict:
            fvalue = resultDict[response_sen1]
        else:
            fvalue = "could not find"
        print("response_sen1--", response_sen1)
        print("final dict", fvalue)
        json_dict = {"ans" : robo_response, "filePath" : fvalue[0], "pageNo" : fvalue[1]}
        return json_dict

In [7]:
#final_list = defaultdict(list)

set_sent_tokens = set()
def sen_dict(clean_txt, final_dict, fname, p_num):
    sent_tokens = nltk.sent_tokenize(clean_txt.lower())# converts to list of sentences
    #print("sent_tokens inside sen_dict",sent_tokens)
    for i in sent_tokens:
        if i in set_sent_tokens:
            final_dict[i].append([fname,p_num])
            final_dict.update(final_dict)
            #print("final_list1",final_dict[i])
            #print("final_list1",final_dict)
        else:
            dict1 = {i : [fname,p_num]}
            final_dict.update(dict1)
            #print("final_dict",final_dict)
            set_sent_tokens.add(i)
        #print("set_sent_tokens",set_sent_tokens)
    return final_dict, sent_tokens    
    #print(final_dict)
    #print("----------------------------------------")

#clean_txt = "cloud. jee. harsha. cloud. dar. cloud."   
#final_dict, sent_tokens = sen_dict(clean_txt, {}, "fname.pdf", 10)
#clean_txt = "cloud."
#print("results here",final_dict)
#final_dict, sent_tokens = sen_dict(clean_txt, final_dict, "chatbot.pptx", 5)
#clean_txt = "dar. cloud."
#sen_dict(clean_txt, {}, "chatbot.pptx", 4)

In [8]:
#converts pdf, returns its text content as a string
def convert(fname, final_dict):
    output = io.StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    page_no = 0
    sent_tokens_all = []
    infile = open(fname, 'rb')
    finalText=""
    #for page in PDFPage.get_pages(infile, pagenums):
    for pageNumber, page in enumerate(PDFPage.get_pages(infile)):
        if pageNumber == page_no:
            interpreter.process_page(page)
            
        converter.close()
        text = output.getvalue()
        clean_txt = clean_text(text)
        result, sent_tokens = sen_dict(clean_txt,final_dict,fname,page_no+1)
        finalText = finalText+text
        sent_tokens_all = sent_tokens_all + sent_tokens
        #print(page_no+1)
        #print(result)
        output.truncate(0)
        output.seek(0)
        #print(finalText)
        page_no += 1
    #print("sent_tokens_all", sent_tokens_all)    
    return finalText, result, sent_tokens_all
    infile.close()

#final_dict = {}
#convert('docs/pdfs/forumQAchatbot.pdf', final_dict)

In [ ]:
corpus_folder = 'docs'
allFileList = getFilesInAllSubFolder(corpus_folder)

final_dict = {}
pdf_sent_tokens_all = []
ppt_sent_tokens_all = []
append_write = 'w'
text_file = open('sample1.txt', append_write)
for filename in allFileList:
    #print(filename)
    
    if filename.endswith(".pdf"):
        try:
            with open('sample1.txt', append_write) as text_file:
                finalText, resultDict, sent_tokens_all = convert(filename, final_dict) #get string of text content of pdf
                #print("inside pdf all",sent_tokens_all)
                pdf_sent_tokens_all = pdf_sent_tokens_all + sent_tokens_all
                text_file.write(finalText)
                #print("inside pdf", resultDict)
            text_file.close()
            pdf_file.close()
        except:
            #print("skipping file due to encoding error "+filename)
            file = open(filename, encoding="iso-8859-1")
            text = file.read()
    elif filename.endswith(".doc") or filename.endswith(".docx"):
        with open('sample1.txt', append_write) as text_file:
            textDoc = textract.process(filename)
            text_file.write(str(textDoc.decode("utf-8")))
            text_file.close()
            #doc = docx.Document(filename)
            # read in each paragraph in file
            #result = [p.text for p in doc.paragraphs]
            #print("inside doc")
            #print(textDoc)
    elif filename.endswith(".pptx"):
        #print(filename)
        page_no = 1
        with open(filename,'rb') as pptx_file, open('sample1.txt', append_write) as text_file:
            prs = Presentation(pptx_file)
            for slide in prs.slides:
                clean_txt = ""
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        text_file.write(shape.text)
                        sent_shape = shape.text
                        if((shape.text).endswith(".")):
                            text_file.write("")
                            sent_shape += ""
                        else:
                            text_file.write(". ")
                            sent_shape += ". "
                        #print(shape.text)
                        clean_txt = clean_txt +" "+ clean_text(shape.text)
                #print("cleaning",clean_txt)
                resultDict, sent_tokens = sen_dict(clean_txt,final_dict,filename,page_no)
                page_no += 1
                ppt_sent_tokens_all = ppt_sent_tokens_all + sent_tokens
            #print("sent_tokens_all_ppts", ppt_sent_tokens_all)
            #print(resultDict)
            #print("______________________________")
            text_file.close()
        pptx_file.close()
    #elif filename.endswith(".xls") or filename.endswith(".xlsx"):
        #with open(filename,'rb') as excel_file, open('sample1.txt', append_write) as text_file:
            #print(text)
            #print("inside ppt")
    if text_file.mode == 'w':
        append_write = 'a'
        resultDict = final_dict
text_file.close()

#a = 'after this, the bot goes on learning from interaction and then it follows that flow of communication, which it had in the past with another user.'

#if a in resultDict:
    #print("resulting dict is-", resultDict[a])

res_sen_list = pdf_sent_tokens_all+ppt_sent_tokens_all
#print("____________res_sen_list__________________")
#print(ppt_sent_tokens_all)
#print(resultDict)


flag=True
while(flag == True):
    user_response = input()
    user_response=user_response.lower()
    result = response(user_response, resultDict, res_sen_list)
    print("result here",result)


how to train a chatbot
response_sen1-- for training our chatbot we also need data; the conversation between a user and a chatbot is the data on which we need to train our models.
final dict ['/Users/jeeva/AlniTek/projects/knowledgebase-bot/docs/pdfs/Building_Chatbots_with_Python.pdf', 146]
result here {'ans': 'for training our chatbot we also need data; the conversation between a user and a chatbot is the data on which we need to train our models.sometimes it becomes difficult to find a dataset freely on the web that fits our own need.we should spend the time we need to get the data gathered.we can ask our friends and family to provide us the sample conversation text of how they would interact with a kind of bot you are building.', 'filePath': '/Users/jeeva/AlniTek/projects/knowledgebase-bot/docs/pdfs/Building_Chatbots_with_Python.pdf', 'pageNo': 146}


In [ ]:
#f=open('sample1.txt','r',errors = 'ignore')
#raw=f.read()
#raw=raw.lower()# converts to lowercase

#sent_tokens = nltk.sent_tokenize(clean_text(raw))# converts to list of sentences

#print("sent_tokens_all_files", sent_tokens)